## 패키지 설치 및 라이브러리 불러오기

In [1]:
pip install pandas scikit-learn sentence-transformers openpyxl


In [2]:
pip install matplotlib

In [3]:
pip install transformers emoji pandas matplotlib accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 9.9 MB/s eta 0:00:00


In [4]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 24.4 MB/s eta 0:00:00


In [5]:
!pip install bertopic sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
import re
import html
import emoji
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from google.colab import drive
import ast
import numpy as np

## 원본 데이터 불러오기

In [ ]:
# drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/2025_문제해결은행/퍼스트밸류/마켓문의_20000건_1.xlsx' #원본 파일 경로 설정
df = pd.read_excel(file_path, header =1)
df.head()


,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000


## 텍스트 전처리

In [ ]:
#텍스트 전처리
def remove_html(text):
    return re.sub(r"<.*?>", " ", text)

def clean_newlines(text):
    return text.replace("\\r", " ").replace("\\n", " ").replace("\n", " ").replace("\t", " ")

def remove_emoji(text):
    return emoji.replace_emoji(text, "")

def remove_url(text):
    return re.sub(r"http\S+|www\S+", "", text)

def remove_special(text):
    # 한글, 영어, 숫자, 감정 표현부호 ?!, . , 만 허용
    return re.sub(r"[^가-힣0-9a-zA-Z?!.,\s]", " ", text)

def clean_spaces(text):
    return ' '.join(text.split())

def remove_greetings(text): #안녕하세요 등 원하는 단어 추가시 제거 가능
    greetings = [
        "첨부이미지"
    ]
    for g in greetings:
        text = text.replace(g, " ")
    return text


def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = remove_html(text)
    text = remove_url(text)
    text = clean_newlines(text)
    text = remove_emoji(text)
    text = remove_special(text)
    text = clean_spaces(text)
    text = remove_greetings(text)
    return text.strip()


In [ ]:
text_col = "inquiry_content"

# 전처리 적용
df["clean_text"] = df[text_col].fillna("").apply(preprocess_text)

,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt,clean_text
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000,안녕하세요 해당 커피 맛있게 먹고있는데 혹시 상품 언제 재입고 되는지 알 수 있을까...
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,어제 제품 받았는데요 4개씩 총 6건 주문했는데 플러스1까지 30개인데요 실제 25...
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,오늘 사은품만 받았어요 본품 4박스는 언제 배송되는지 확인 부탁드립니다
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,3번 세트 4개 구입했는데요. 오늘..사은품인 녹용골드스틱만 오고 배송완료라고 떴어...
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,300포 주문했는데..30포가 왔네요? 큰박스에 달랑 세 뭉치 들어 깜놀이었네요.....


## 모델 로드
- 라벨이 없는 원본 문의 데이터를 분석하기 위해, 사전학습된 감성 분석 모델 로드하여 사용
- 해당 모델은 총 60가지 세부 감정으로 텍스트 분류 / 단순 긍/부정보다 더 정교한 감정 파악 가능
- 문의글 특성상 단순 긍정/부정 분류가 잘 맞지 않기 때문에, 먼저 60개 감정 중 어떤 감정인지 분류한 뒤
→ 각 감정을 긍정/부정 범주에 다시 매핑하는 방식으로 최종 감성 결과를 도출

In [ ]:
#https://huggingface.co/hun3359/klue-bert-base-sentiment 사용
model_name = "hun3359/klue-bert-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 감성분석

In [ ]:
def predict_sentiments(texts, batch_size=16):
    """
    입력된 텍스트 리스트에 대해 사전학습된 감성 분류 모델로 예측을 수행하는 함수.

    - 텍스트를 batch_size 단위로 나누어 GPU/CPU 환경에서 추론
    - 토크나이저로 입력을 텐서 형태로 변환(padding/truncation 적용)
    - 모델의 로짓(logits)에서 argmax를 취해 클래스 ID(예측 라벨) 도출
    - 모든 배치의 예측 결과를 하나의 리스트로 반환

    Args:
        texts (list[str]): 감성을 예측할 텍스트 리스트
        batch_size (int): 한 번에 추론할 배치 크기 (기본값 16)

    Returns:
        list[int]: 각 텍스트에 대한 예측 감정 라벨 ID 목록
    """
    results = []
    device = "cuda" if torch.cuda.is_available() else "cpu"

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

        results.extend(preds)
    return results


In [ ]:
# 예측 결과(0~59)의 정수 ID를 60개 감정 라벨명으로 매핑하는 딕셔너리 확인
model.config.id2label

{0: '분노',
 1: '툴툴대는',
 2: '좌절한',
 3: '짜증내는',
 4: '방어적인',
 5: '악의적인',
 6: '안달하는',
 7: '구역질 나는',
 8: '노여워하는',
 9: '성가신',
 10: '슬픔',
 11: '실망한',
 12: '비통한',
 13: '후회되는',
 14: '우울한',
 15: '마비된',
 16: '염세적인',
 17: '눈물이 나는',
 18: '낙담한',
 19: '환멸을 느끼는',
 20: '불안',
 21: '두려운',
 22: '스트레스 받는',
 23: '취약한',
 24: '혼란스러운',
 25: '당혹스러운',
 26: '회의적인',
 27: '걱정스러운',
 28: '조심스러운',
 29: '초조한',
 30: '상처',
 31: '질투하는',
 32: '배신당한',
 33: '고립된',
 34: '충격 받은',
 35: '가난한 불우한',
 36: '희생된',
 37: '억울한',
 38: '괴로워하는',
 39: '버려진',
 40: '당황',
 41: '고립된(당황한)',
 42: '남의 시선을 의식하는',
 43: '외로운',
 44: '열등감',
 45: '죄책감의',
 46: '부끄러운',
 47: '혐오스러운',
 48: '한심한',
 49: '혼란스러운(당황한)',
 50: '기쁨',
 51: '감사하는',
 52: '신뢰하는',
 53: '편안한',
 54: '만족스러운',
 55: '흥분',
 56: '느긋',
 57: '안도',
 58: '신이 난',
 59: '자신하는'}

In [ ]:
#감성 분석 함수 적용.
texts = df["clean_text"].tolist() #리스트로 변경 후 input
df["sentiment_raw"] = predict_sentiments(texts)
df.head()

,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt,clean_text,sentiment_raw
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000,안녕하세요 해당 커피 맛있게 먹고있는데 혹시 상품 언제 재입고 되는지 알 수 있을까...,6
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,어제 제품 받았는데요 4개씩 총 6건 주문했는데 플러스1까지 30개인데요 실제 25...,29
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,오늘 사은품만 받았어요 본품 4박스는 언제 배송되는지 확인 부탁드립니다,56
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,3번 세트 4개 구입했는데요. 오늘..사은품인 녹용골드스틱만 오고 배송완료라고 떴어...,57
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,300포 주문했는데..30포가 왔네요? 큰박스에 달랑 세 뭉치 들어 깜놀이었네요.....,40


In [ ]:
#결과물 엑셀로 다운로드
# df.to_excel('감성분석결과물.xlsx') 

# 감성 분석 결과물

In [1]:
#구글 드라이브와 연동
# drive.mount('/content/drive')

In [5]:
# df_emo = pd.read_excel('/content/감성분석결과물1117.xlsx')
#저장된 위 감성분석 결과물 불러오기
import pandas as pd

df_emo = pd.read_excel(r"C:\Users\윤서현\Desktop\문제해결은행\퍼스트밸류\정리\감성분석결과물1117.xlsx")

In [6]:
# 감정 라벨 매핑 딕셔너리
emotion_map = {
    0: '분노', 1: '툴툴대는', 2: '좌절한', 3: '짜증내는', 4: '방어적인',
    5: '악의적인', 6: '안달하는', 7: '구역질 나는', 8: '노여워하는', 9: '성가신',
    10: '슬픔', 11: '실망한', 12: '비통한', 13: '후회되는', 14: '우울한',
    15: '마비된', 16: '염세적인', 17: '눈물이 나는', 18: '낙담한', 19: '환멸을 느끼는',
    20: '불안', 21: '두려운', 22: '스트레스 받는', 23: '취약한', 24: '혼란스러운',
    25: '당혹스러운', 26: '회의적인', 27: '걱정스러운', 28: '조심스러운', 29: '초조한',
    30: '상처', 31: '질투하는', 32: '배신당한', 33: '고립된', 34: '충격 받은',
    35: '가난한 불우한', 36: '희생된', 37: '억울한', 38: '괴로워하는', 39: '버려진',
    40: '당황', 41: '고립된(당황한)', 42: '남의 시선을 의식하는', 43: '외로운', 44: '열등감',
    45: '죄책감의', 46: '부끄러운', 47: '혐오스러운', 48: '한심한', 49: '혼란스러운(당황한)',
    50: '기쁨', 51: '감사하는', 52: '신뢰하는', 53: '편안한', 54: '만족스러운',
    55: '흥분', 56: '느긋', 57: '안도', 58: '신이 난', 59: '자신하는'
}

# sentiment_raw → 감정 텍스트로 매핑한 새 열 생성
df_emo["sentiment_label"] = df_emo["sentiment_raw"].map(emotion_map)


In [7]:
# 49 이하 = 부정(negative), 50 이상 = 긍정(positive)으로 나누기
df_emo["sentiment_binary"] = df_emo["sentiment_raw"].apply(
    lambda x: "부정" if x <= 49 else "긍정"
)

In [ ]:
#매핑한 결과물 엑셀 파일로 저장
# df_emo.to_excel('감정텍스트라벨링.xlsx', index = False)

# 부정 문의글 토픽 파악

In [8]:
df_emo.columns

Index(['Unnamed: 0', 'market_account_no', 'market_inquiry_no',
       'inquiry_type_cd', 'inquiry_type', 'inquiry_title', 'inquiry_content',
       'inquiry_date', 'reply_yn', 'inquiry_raw', 'update_dt', 'create_dt',
       'clean_text', 'sentiment_raw', 'sentiment_label', 'sentiment_binary'],
      dtype='object')

In [9]:
#부정만 추출
neg_df = df_emo[df_emo['sentiment_binary'] == '부정'].copy()
texts = neg_df['clean_text'].astype(str).tolist()

In [11]:
#규칙기반 내용 분류
import re

def categorize_multilabel(text):
    text = str(text)
    labels = []

    # 배송 관련
    if re.search("배송|늦|빠르|도착|택배|출고|누락|안왔|오배송|없음|미도착|발송", text):
        labels.append("배송")

    # 제품 불량
    if re.search("불량|깨짐|찢|파손|오염|이상|스크래치|찌그러짐", text):
        labels.append("제품 불량")

    # 사은품
    if re.search("사은품|증정|서비스", text):
        labels.append("사은품")

    # 유통기한
    if re.search("유통기한|상함|상했|날짜|신선도", text):
        labels.append("유통기한")

    # 재고/품절/선택 옵션
    if re.search("품절|재고|옵션|선택|색상|색깔|모델|재입고", text):
        labels.append("옵션/재고")


    # 환불 및 교환
    if re.search("환불|교환|취소|반품", text):
        labels.append("환불 및 교환")

    #가격/할인/결제
    if re.search("가격|비싸|저렴|쿠폰|할인|포인트|결제|카드|영수증", text):
      labels.append("가격/할인/결제")

    #용량/수량/누락
    if re.search("수량|용량|부족|더왔|덜왔|구성", text):
        labels.append("수량/구성")

    #제품 문의
    if re.search('사이즈', text):
        labels.append('제품 문의')



    if len(labels) == 0:
        labels.append("기타")

    return labels

neg_df['categories'] = neg_df['clean_text'].apply(categorize_multilabel)


In [ ]:
# 1) '기타' 포함된 텍스트만 추출
df_etc_any = neg_df[neg_df['categories'].apply(lambda x: "기타" in x)]
df_etc_any_text = df_etc_any['clean_text'].tolist()

# 2) BERTopic 모델 학습
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

#https://huggingface.co/jhgan/ko-sroberta-multitask
embedding_model = SentenceTransformer("jhgan/ko-sroberta-multitask")
topic_model = BERTopic(embedding_model=embedding_model, min_topic_size=20)

topics, probs = topic_model.fit_transform(df_etc_any_text)

# 3) 토픽 결과를 df_etc_any에 붙이기
df_etc_any = df_etc_any.copy()
df_etc_any['topic'] = topics
df_etc_any['topic_prob'] = probs

# 4) 토픽 정보 테이블
topic_info = topic_model.get_topic_info()
print(topic_info.head())

# 5) 원본 neg_df에 토픽 번호 붙이기 (기타 아닌 애들은 NaN)
neg_df = neg_df.copy()
neg_df['topic'] = None

# df_etc_any의 index 기준으로 토픽 매핑
neg_df.loc[df_etc_any.index, 'topic'] = df_etc_any['topic']


   Topic  Count                  Name  \
0     -1    726    -1_혹시_안녕하세요_언제_왔어요   
1      0    243     0_아직_아직도_받지_못했습니다   
2      1    138       1_4팩만_4팩_왔어요_8팩   
3      2    126        2_1번_xl_3번_인가요   
4      3    114  3_한박스만_왔어요_하나만_주문했는데   

                                      Representation  \
0  [혹시, 안녕하세요, 언제, 왔어요, 있나요, 주문했는데, 언제쯤, 상품, 부탁드립...   
1  [아직, 아직도, 받지, 못했습니다, 미수령, 물건을, 연락이, 연락도, 물건이, 확인]   
2  [4팩만, 4팩, 왔어요, 8팩, 4팩은, 4팩만왔어요, 왔습니다, 8팩인데, 4개...   
3  [1번, xl, 3번, 인가요, 되나요, 남자, 가슴둘레, 레귤러핏인가요, 입는데,...   
4  [한박스만, 왔어요, 하나만, 주문했는데, 왔네요, 1개만, 1박스만, 확인부탁드립...   

                                 Representative_Docs  
0  [안녕하세요 본 상품 4T로 구매하였는데 혹시 5T로 변경 가능할까요??, 언제 받...  
1  [물건을 아직 못 받았습니다, 11일에 주문하여 아직 받지 못했습니다. 11일에 주...  
2  [안녕하세요 8팩 주문했는데 4팩만 왔어요 확인부탁드려요, 4 4팩 시켰는데 점보 ...  
3  [1번 네이비 xl 총 길이와 어깨 길이 어떻게 되나요?, 1번 상품 XL 총길이 ...  
4  [두개 주문했는데 하나만 왔어요 하나 더 보내주세요, 두박스 주문했는데 한박스만 왔...  


In [ ]:
#토픽 info 엑셀로 다운로드
# topic_info.to_excel('11271253topicinfo.xlsx', index = False)

In [ ]:
#토픽 분류된 데이터 다운로드
# neg_df.to_excel('11271253negdf.xlsx', index = False)

In [ ]:
#분류된 토픽별 토픽 부여
topic_map = {
    -1: "제품 문의",
    0: "배송",
    1: "배송",
    2: "제품 문의",
    3: "배송",
    4: "제품 불량",
    5: "배송",
    6: "제품 문의",
    7: "제품 문의",
    8: "배송",
    9: "노이즈",
    10: "배송",
    11: "제품 불량",
    12: "요청사항",
    13: "연락 확인 요청",
    14: "제품 문의",
    15: "배송",
    16: "제품 문의",
    17: "요청사항",
    18: "고객센터 (G마켓, 위메프 등)",
    19: "배송",
    20: "배송",
    21: "노이즈",
    22: "가격/할인/결제,옵션/재고",
    23: "가격/할인/결제",
    24: "배송",
    25: "제품 문의",
    26: "제품 문의",
    27: "노이즈",
    28: "제품 문의",
    29: "제품 문의"
}


In [ ]:
import ast
import numpy as np

# 함수 정의
def map_category(row):
    topic = row['topic']

    # 1) topic이 None or NaN 이면 categories 사용
    if pd.isna(topic):
        raw = row['categories']

        # categories가 문자열인 경우 리스트로 변환
        if isinstance(raw, str):
            try:
                parsed = ast.literal_eval(raw)
                return parsed    # 리스트 반환
            except:
                return raw       # 변환 실패 시 문자열 그대로 반환
        else:
            return raw

    # 2) topic이 숫자면 topic_map 매핑
    else:
        topic_int = int(topic)
        return topic_map.get(topic_int, "기타")  # 혹시 매핑 안 된 숫자 대비

# 3) 최종 결과 컬럼 생성
neg_df['final_category'] = neg_df.apply(map_category, axis=1)


In [ ]:
#라벨 작업 완료 파일 엑셀로 다운로드
# neg_df.to_excel('라벨 작업 완료한 파일.xlsx', index = False)

# 최종 파일 병합 및 정리

In [70]:
# 1) 긍정 데이터만 필터링
df_pos = df_emo[df_emo['sentiment_binary'] == '긍정'].copy()

# 2) 긍정 데이터에 final_category 컬럼 추가
#    (긍정 문장은 토픽 분류 대상이 아니므로 기본 라벨 부여)
df_pos['final_category'] = '긍정'

# 3) neg_df와 df_pos를 세로로 병합
final_df = pd.concat([neg_df, df_pos], axis=0, ignore_index=True)

# 4) 필요하면 타입 안정화 (optional)
final_df = final_df.reset_index(drop=True)
final_df = final_df.dropna(subset=['clean_text']) #문의글 빈 값 지우기

In [ ]:
#리스트 형태 문자열 형태로 바꾸기
final_df['final_category'] = final_df['final_category'].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)

In [ ]:
#최종 결과물 엑셀로 다운로드
final_df.to_excel('최종결과물.xlsx', index = False)

### (감성 분석 모델 성능 테스트)

In [ ]:
test_sentences = [
    "너무 좋아요 완전 만족합니다",     # ➜ 긍정
    "그냥 그렇네요 보통이에요",         # ➜ 중립
    "진짜 짜증나요 불량이에요 최악입니다"  # ➜ 부정
]

inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

print(outputs.logits)
print(torch.argmax(outputs.logits, dim=1))


tensor([[-0.3189, -0.1398, -0.9447,  0.1076, -0.7067, -0.2232, -1.0427, -0.2145,
         -0.8921, -0.4748, -0.3166, -0.4429, -1.7091, -0.4980, -0.5093, -0.1659,
         -0.4872,  0.9398, -0.5769, -1.3159, -0.1166, -1.4747, -1.3910, -0.8999,
         -0.3236, -0.8831, -0.6081, -0.5856, -1.1859, -0.4765, -0.8731, -0.7028,
          0.1720, -0.1601, -1.1010, -0.9688, -0.4001, -0.0832, -0.9372,  0.5597,
         -0.5697, -1.1306, -0.7737, -1.0172, -1.6481, -0.4134,  0.8612, -1.5997,
         -2.2341, -0.6054,  3.5421,  3.0593,  3.3420,  3.8690,  7.9563,  2.9380,
          3.3752,  2.7907,  2.6059,  1.3990],
        [ 1.0794,  0.5159, -0.1824,  0.5582,  2.1587,  1.1126, -1.9579,  0.6401,
         -0.0539,  0.2482, -0.0891,  0.0392, -1.2603, -0.3791, -0.5736,  0.5870,
          0.6443, -0.5374,  0.5532,  0.8419, -0.4262, -0.2564, -0.8441, -0.0905,
          0.0637,  0.4707,  1.0449, -0.5539, -0.3395, -1.0479, -0.1796,  0.5232,
          0.2009,  0.8385, -0.4621, -0.9636, -0.3959,  0.5908, 